# Import Library

In [8]:
from pythainlp.tokenize import word_tokenize
from tqdm import tqdm
import re

# Remove link and replace with keyword
Remove any string seperate by whitespace that begin with http:// or https:// and replace it with "<LINK>" keyword

In [2]:
with_link = []

def replace_link(idx, text, replace_text = '<LINK>'):
    new_text = text
    pattern = re.compile("http[s]{0,1}://")
    if pattern.search(new_text):
        with_link.append(idx)
        new_text = []
        for w in text.split(" "):
            if pattern.search(w):
                new_text += [replace_text]
            else:
                new_text += [w]
        new_text = " ".join(new_text)
    return new_text

# Define constant parameters

In [14]:
inputFile = 'sentences-30k.txt'
outputFile = 'tokenized_out-30k-space.txt'

# Open input file

In [10]:
data = []
idx = 0

with open(inputFile, 'r') as file:
    for line in file:
        if len(line.strip()) > 0:
            data.append(replace_link(idx, re.sub(r'[5]{2,}', '555', line.strip())))
            idx += 1

In [11]:
# Tokenize
words = set()
tokenized = []
for line in tqdm(data):
    tokenized.append(word_tokenize(line))
    words = set(tokenized[-1]) | words

100%|██████████| 35434/35434 [00:22<00:00, 1586.42it/s]


In [12]:
new_words = set()
new_tokenized = []
for idx, line in enumerate(tokenized):
    newTokenize = []
    if idx in with_link:
        for w in line:
            if '<LINK>' in w.split(' '):
                newTokenize += ['<LINK>']
                if len(w.split(' ')) > 1:
                    newTokenize += [' ']
                    ttmp = w.split(' ')[1:]
                    newTokenize += word_tokenize(" ".join(ttmp), engine='newmm')
            else:
                newTokenize += word_tokenize(w, engine='newmm')
    else:
        for w in line:
            newTokenize += word_tokenize(w, engine='newmm')
    
    new_tokenized.append(newTokenize)
    new_words = new_words | set(newTokenize)

# Writing output file

In [15]:
with open(outputFile, 'w') as file:
    for line in new_tokenized:
        file.write("{}\n".format(" ".join(line)))